# **Review Classification and Topic Extraction using Google NLP**
This notebook uses Google Cloud NLP to classify reviews as suggestions or not, and extract topics from them.

In [3]:
!pip install google-cloud-language pandas

In [9]:
import os
import pandas as pd
import numpy as np
from google.cloud import language_v1
from google.cloud.language_v1 import types
import json

In [10]:
reviews = pd.read_csv('newest_gm_reviews.csv')
reviews

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user
0,ChZDSUhNMG9nS0VJQ0FnTURndlByUlFREAE,The best,5 days ago,2025-02-28 17:21:31.818351,5.0,coolguy78521,0,https://www.google.com/maps/contrib/1082486407...,NaN
1,ChZDSUhNMG9nS0VJQ0FnTURnbk51Skx3EAE,Good,5 days ago,2025-02-28 17:21:31.819834,5.0,Yamil Hurtado,0,https://www.google.com/maps/contrib/1055654216...,NaN
2,ChdDSUhNMG9nS0VJQ0FnTURnbk5PUDNRRRAB,Excellent pizza in NEW YORK and service,5 days ago,2025-02-28 17:21:31.820172,5.0,ABEL YAMIL HURTADO MAZON,1,https://www.google.com/maps/contrib/1090846354...,NaN
3,ChZDSUhNMG9nS0VJQ0FnTURnN0lDZlpBEAE,Ver delicious and made me full. The staff were...,5 days ago,2025-02-28 17:21:31.820502,5.0,انس الشغدري,21,https://www.google.com/maps/contrib/1035613830...,NaN
4,ChZDSUhNMG9nS0VJQ0FnTURnaUltLWRBEAE,GREAT pizza and dessert!!! We can’t back twice...,6 days ago,2025-02-28 17:21:31.821153,5.0,Leticia Dotson,0,https://www.google.com/maps/contrib/1180842954...,NaN
...,...,...,...,...,...,...,...,...,...
495,ChdDSUhNMG9nS0VJQ0FnSURGcC1fbW93RRAB,NaN,a year ago,2025-02-28 17:26:01.475583,5.0,ito ayami,19,https://www.google.com/maps/contrib/1029892739...,NaN
496,ChZDSUhNMG9nS0VJQ0FnSURGLS1pd2V3EAE,NaN,a year ago,2025-02-28 17:26:01.476054,5.0,Jesus Robledo,9,https://www.google.com/maps/contrib/1109079903...,NaN
497,ChdDSUhNMG9nS0VJQ0FnSURGaTdpUWxRRRAB,NaN,a year ago,2025-02-28 17:26:01.476349,4.0,Mariella Quispe,0,https://www.google.com/maps/contrib/1149462966...,NaN
498,ChZDSUhNMG9nS0VJQ0FnSURGczhQRkRnEAE,The best turkey burger,a year ago,2025-02-28 17:26:01.476649,5.0,Wayne Francis,0,https://www.google.com/maps/contrib/1025865150...,NaN


## **Step 2: Authenticate Google NLP API**

In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './service-account-key.json'

client = language_v1.LanguageServiceClient()

## **Step 3: Classify Reviews as Suggestions or Not with IS_SUGGESTION**


In [34]:
def is_suggestion(reviews):
    document = language_v1.Document(content=reviews, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment 
    if -0.2 <= sentiment.score <= 0.2:
        return "is suggestion"
    else:
        return "not suggestion"

reviews['caption'] = reviews['caption'].fillna("")  
reviews['suggestion_status'] = reviews['caption'].apply(is_suggestion)
reviews
suggestion_count = (reviews['suggestion_status'] == 'is suggestion').sum()
not_suggestion_count = (reviews['suggestion_status'] == 'not suggestion').sum()

## Step 4: Extracting Topics from Reviews

In [33]:
def extract_topics(reviews):
    document = language_v1.Document(content=reviews, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_entities(request={'document': document})
    topics = []
    for entity in response.entities[:3]:
        topics.append(entity.name)
    if topics:
        return ', '.join(topics) 
    else:
        return 'No topics found'

reviews['topics'] = reviews['caption'].apply(extract_topics)
reviews

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user,suggestion_status,topics
0,ChZDSUhNMG9nS0VJQ0FnTURndlByUlFREAE,The best,5 days ago,2025-02-28 17:21:31.818351,5.0,coolguy78521,0,https://www.google.com/maps/contrib/1082486407...,NaN,not suggestion,best
1,ChZDSUhNMG9nS0VJQ0FnTURnbk51Skx3EAE,Good,5 days ago,2025-02-28 17:21:31.819834,5.0,Yamil Hurtado,0,https://www.google.com/maps/contrib/1055654216...,NaN,not suggestion,No topics found
2,ChdDSUhNMG9nS0VJQ0FnTURnbk5PUDNRRRAB,Excellent pizza in NEW YORK and service,5 days ago,2025-02-28 17:21:31.820172,5.0,ABEL YAMIL HURTADO MAZON,1,https://www.google.com/maps/contrib/1090846354...,NaN,not suggestion,"pizza, service, NEW YORK"
3,ChZDSUhNMG9nS0VJQ0FnTURnN0lDZlpBEAE,Ver delicious and made me full. The staff were...,5 days ago,2025-02-28 17:21:31.820502,5.0,انس الشغدري,21,https://www.google.com/maps/contrib/1035613830...,NaN,not suggestion,"place, staff, faces"
4,ChZDSUhNMG9nS0VJQ0FnTURnaUltLWRBEAE,GREAT pizza and dessert!!! We can’t back twice...,6 days ago,2025-02-28 17:21:31.821153,5.0,Leticia Dotson,0,https://www.google.com/maps/contrib/1180842954...,NaN,not suggestion,"pizza, dessert, rating"
...,...,...,...,...,...,...,...,...,...,...,...
495,ChdDSUhNMG9nS0VJQ0FnSURGcC1fbW93RRAB,,a year ago,2025-02-28 17:26:01.475583,5.0,ito ayami,19,https://www.google.com/maps/contrib/1029892739...,NaN,is suggestion,No topics found
496,ChZDSUhNMG9nS0VJQ0FnSURGLS1pd2V3EAE,,a year ago,2025-02-28 17:26:01.476054,5.0,Jesus Robledo,9,https://www.google.com/maps/contrib/1109079903...,NaN,is suggestion,No topics found
497,ChdDSUhNMG9nS0VJQ0FnSURGaTdpUWxRRRAB,,a year ago,2025-02-28 17:26:01.476349,4.0,Mariella Quispe,0,https://www.google.com/maps/contrib/1149462966...,NaN,is suggestion,No topics found
498,ChZDSUhNMG9nS0VJQ0FnSURGczhQRkRnEAE,The best turkey burger,a year ago,2025-02-28 17:26:01.476649,5.0,Wayne Francis,0,https://www.google.com/maps/contrib/1025865150...,NaN,not suggestion,turkey burger
